Final Project

In [21]:
import plotly.express as px
import pycountry
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import pandas as pd


In [22]:
df = pd.read_csv('CO2_Emissions_1960-2018.csv')

# reshape the date to be able to display co2 emmsion per years
df = df.melt(id_vars=["Country Name"], var_name="Year", value_name="Emission Per Year")

df.head()

,Country Name,Year,Emission Per Year
0,Aruba,1960,204.631696
1,Africa Eastern and Southern,1960,0.906060
2,Afghanistan,1960,0.046057
3,Africa Western and Central,1960,0.090880
4,Angola,1960,0.100835


In [ ]:

# ISO-Code-Funktion anwenden
def get_iso_alpha_3(country_name):
    try:
        return pycountry.countries.lookup(country_name).alpha_3
    except LookupError:
        return None

df['iso_alpha'] = df['Country Name'].apply(get_iso_alpha_3)

app = dash.Dash(__name__)

app.layout = html.Div([
    # create worldmap
    dcc.Graph(
        id='choropleth-map',
        figure=px.choropleth(
            df,
            locations="iso_alpha",
            color="Emission Per Year",
            scope="world",
            animation_frame="Year",
            hover_name="Country Name",
            width=1000,
            height=600
        )
    ),
    dcc.Graph(id='bar-chart')
])

@app.callback(
    Output('bar-chart', 'figure'),
    [Input('choropleth-map', 'clickData')]
)

## prepare bar chart after cllick on country
def update_bar_chart(clickData):
    if clickData is None:
        country_name = 'Switzerland'
    else:
        country_name = clickData['points'][0]['hovertext']
    filtered_df = df[df['Country Name'] == country_name]
    fig = px.bar(filtered_df, x='Year', y='Emission Per Year', title=f'CO2 Emissions for {country_name}')
    return fig

if __name__ == '__main__':
    app.run_server(debug=True)
